In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess

NUM_PROGRAM = 8

cut_programs = np.load('cut_Programs.npy')
cut_questions = np.load('cut_Questions.npy')

voc_dict = np.load('voc_dict.npy')

#print(cut_programs[0][0])
sentences = []
tag = 0
for program in cut_programs[:1]:
    for episode in program[:1]:
        for sentence in episode[:5]:
            sentences.append(TaggedDocument(sentence, [tag]))
            tag = tag+1
            
#print(sentences[:5])         

In [4]:
print(cut_questions[0][0])

# 6 optional reponses
for i in range(1, 7):
    print(cut_questions[0][i])

[['媽給', '你', '送', '錢包', '來', '啦', ' ', '來', ' ', '你', '看', '一下', '是', '不', '是', '這個'], ['對', ' ', '就是', '這個', ' ', '你', '在', '哪裡', '找到', '它', '的'], []]
['你', '看', ' ', '這是', '我', '新', '買', '的', '錢包']
['我', '的', '錢包', '不見了', '啦']
['以後', '上', '網咖', '的', '錢包', '在', '我', '身上']
['什麼', '有', '錢包', '場']
['早上', '你', '爸爸', '在', '車上', '找到', '的', ' ', '一定', '是', '前天', '你', '放學', '的', '時候', '掉', '在', '車上', '了']
['我', '為什麼', '要給', '你們', '錢包']


In [5]:
import random

NUM_TRAIN = 10000
TRAIN_VALID_RATE = 0.7
def generate_training_data():
    Xs, Ys = [], []
    
    for i in range(NUM_TRAIN):
        pos_or_neg = random.randint(0, 1)
        
        if pos_or_neg==1:
            program_id = random.randint(0, NUM_PROGRAM-1)
            episode_id = random.randint(0, len(cut_programs[program_id])-1)
            line_id = random.randint(0, len(cut_programs[program_id][episode_id])-2)
            
            Xs.append([cut_programs[program_id][episode_id][line_id], 
                       cut_programs[program_id][episode_id][line_id+1]])
            Ys.append(1)
            
        else:
            first_program_id = random.randint(0, NUM_PROGRAM-1)
            first_episode_id = random.randint(0, len(cut_programs[first_program_id])-1)
            first_line_id = random.randint(0, len(cut_programs[first_program_id][first_episode_id])-1)
            
            second_program_id = random.randint(0, NUM_PROGRAM-1)
            second_episode_id = random.randint(0, len(cut_programs[second_program_id])-1)
            second_line_id = random.randint(0, len(cut_programs[second_program_id][second_episode_id])-1)
            
            Xs.append([cut_programs[first_program_id][first_episode_id][first_line_id], 
                       cut_programs[second_program_id][second_episode_id][second_line_id]])
            Ys.append(0)
    
    return Xs, Ys
Xs, Ys = generate_training_data()

x_train, y_train = Xs[:int(NUM_TRAIN*TRAIN_VALID_RATE)], Ys[:int(NUM_TRAIN*TRAIN_VALID_RATE)]
x_valid, y_valid = Xs[int(NUM_TRAIN*TRAIN_VALID_RATE):], Ys[int(NUM_TRAIN*TRAIN_VALID_RATE):]

In [6]:
example_doc = []

# lines in 1st episode in program 0 
for line in cut_programs[0][0]:
    example_line = ''
    for w in line:
        if w in voc_dict:
            example_line += w+' '
        
    example_doc.append(example_line)

print(example_doc)

['還好 天氣 不錯 ', '昨天 晚上 ', '看到 很多 流星 ', '這次 收穫 真 豐富 ', '當然 豐富 啦 ', '說 嘛 ', '精心 製作 ', '被 一個 人 吃掉 ', '真的 嗎 ', '不要 忘記 做 秘密 檔案 ', '', '先 去 看 工具 ', '先來 看 花 ', '美麗 細心 賢賢 ', '最 喜歡 這些 花花草草 ', '而且 ', '三個 人 裡面 唯一 女生 ', '所以 平常 他們 最 喜歡 叫 ', '一朵 美麗 小花 ', '任何 事情 充滿 好奇心 ', '而且 喜歡 搞笑 阿輝 ', '對於 電腦 還有 電器 用品 ', '一 把 罩 ', '所以 他們 又 叫 . ', '就是 好用 拼 第一 小兵 ', '平常 喜歡 ', '做 一些 DIY ', '為了 每 次要 去 探索 ', '百米 ', '以 體力 ', '所以 他們 叫 瘋狂 百米 ', '三個 湊 在一起 最 喜歡 ', '探索 ', '', '台灣 走透透 ', '上 至 天上 飛 小鳥 ', '下 到 地上 爬 蟲蟲 ', '探索 對象 ', '這裡 秘密 基地 ', '噓 小聲 一點 ', '想到 ', '倉庫 還有 這 電腦 ', '豆油伯 借給 ', '太 吵 話 ', '他 到時候 會不會 不想 借給 ', '不會 啦 豆油伯 人 最好 ', '看 ', '他給 倉庫 真的 很棒 ', '可以 裡面 ', '做 任何 想 做 事情 ', '而且 還可以 常常 發現 特別 寶物 ', '所以 ', '每次 探索 回來 所 做 秘密 檔案 ', '可以 放 倉庫 ', '也 就是 秘密 基地 裡 ', '說 到 這裡 ', '謝謝 豆油伯 ', '謝謝 ', '不過 講到 說 探索 ', '到 哪裡 去 探索 ', '只要 大家 下課 後 ', '鎖定 公共 電視 ', '可以 看到 美麗 小花 ', '還有 阿輝 . ', '還有 小兵 ', '人稱 瘋狂 百米 ', '三人 組合 而成 ', '「 下課 花路 米 」 ', '那 接下來 ', '探索 怎麼樣 主題 ', '想想 看 ', '喂 小兵 ', '人家 想想 看 ', '一直 那邊 ', '', '我剛 說 那麼多 話 ', '口 渴 ', '而且 又 猴子 ', '幹嘛 

In [7]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(sentences)